**PROJETO WALMART**

**Importação bibliotecas**

In [48]:
import pandas as pd

In [49]:
from sqlalchemy import create_engine
import psycopg2

In [50]:
df = pd.read_csv('Walmart_dataset.csv')

**Data exploration e Data cleaning**

In [51]:
df.shape # quantidade de colunas e linhas

(10051, 11)

In [52]:
df.columns # título de cada coluna

Index(['invoice_id', 'Branch', 'City', 'category', 'unit_price', 'quantity',
       'date', 'time', 'payment_method', 'rating', 'profit_margin'],
      dtype='object')

In [53]:
df.head() # primeiras 5 linhas de cada coluna

,invoice_id,Branch,City,category,unit_price,quantity,date,time,payment_method,rating,profit_margin
0,1,WALM003,San Antonio,Health and beauty,$74.69,7.0,05/01/19,13:08:00,Ewallet,9.1,0.48
1,2,WALM048,Harlingen,Electronic accessories,$15.28,5.0,08/03/19,10:29:00,Cash,9.6,0.48
2,3,WALM067,Haltom City,Home and lifestyle,$46.33,7.0,03/03/19,13:23:00,Credit card,7.4,0.33
3,4,WALM064,Bedford,Health and beauty,$58.22,8.0,27/01/19,20:33:00,Ewallet,8.4,0.33
4,5,WALM013,Irving,Sports and travel,$86.31,7.0,08/02/19,10:37:00,Ewallet,5.3,0.48


*Verificação de dados faltantes*

In [54]:
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10051 entries, 0 to 10050
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   invoice_id      10051 non-null  int64  
 1   Branch          10051 non-null  object 
 2   City            10051 non-null  object 
 3   category        10051 non-null  object 
 4   unit_price      10020 non-null  object 
 5   quantity        10020 non-null  float64
 6   date            10051 non-null  object 
 7   time            10051 non-null  object 
 8   payment_method  10051 non-null  object 
 9   rating          10051 non-null  float64
 10  profit_margin   10051 non-null  float64
dtypes: float64(3), int64(1), object(7)
memory usage: 863.9+ KB


In [55]:
df.isnull().sum()  # quantidade de dados faltantes por coluna

invoice_id         0
Branch             0
City               0
category           0
unit_price        31
quantity          31
date               0
time               0
payment_method     0
rating             0
profit_margin      0
dtype: int64

*Exclusão de dados ausentes*

In [56]:
Cdf = df.copy() # criação de uma cópia para ser alterada

In [57]:
# Excluindo dados faltantes 
Cdf = Cdf.dropna()

In [58]:
# Verificando se os daos ausentes foram excluídos
Cdf.isnull().sum() 

invoice_id        0
Branch            0
City              0
category          0
unit_price        0
quantity          0
date              0
time              0
payment_method    0
rating            0
profit_margin     0
dtype: int64

In [59]:
Cdf.shape

(10020, 11)

*Verificação de duplicatas*

In [60]:
Cdf.duplicated().sum() # verificando exitência de duplicatas

np.int64(51)

*Exclusão de duplicatas*

In [61]:
Cdf.drop_duplicates(inplace=True) # Excluindo duplicatas

In [62]:
#Verificando se as duplicatas foram excluídas
Cdf.duplicated().sum()

np.int64(0)

In [63]:
Cdf.shape

(9969, 11)

*Verificação dos tipos dos dados*

In [64]:
Cdf.dtypes

invoice_id          int64
Branch             object
City               object
category           object
unit_price         object
quantity          float64
date               object
time               object
payment_method     object
rating            float64
profit_margin     float64
dtype: object

*Padronização dos tipos de dados*

In [65]:
# Convertando o tipo de dado da coluna 'quantity' de float para int 
Cdf['quantity'] = Cdf['quantity'].astype(int)

In [66]:
# Removendo $ da coluna'unit_price' para converter os dados para float
Cdf [ 'unit_price' ] = df [ 'unit_price' ].str.replace('$' , ' ')

In [67]:
# convertendo os dados da coluna 'quanunit_price' de object para float
Cdf['unit_price'] = Cdf['unit_price'].astype(float)

In [68]:
# Verificando a conversão de dados
Cdf.dtypes

invoice_id          int64
Branch             object
City               object
category           object
unit_price        float64
quantity            int64
date               object
time               object
payment_method     object
rating            float64
profit_margin     float64
dtype: object

*Criando nova coluna*

In [69]:
# Criando coluna de preço total 
Cdf['total_price'] = Cdf['unit_price'] * Cdf['quantity']

In [70]:
# Verificando a criação da nova coluna: 'total_price'
Cdf.columns

Index(['invoice_id', 'Branch', 'City', 'category', 'unit_price', 'quantity',
       'date', 'time', 'payment_method', 'rating', 'profit_margin',
       'total_price'],
      dtype='object')

In [71]:
# Verificando a nova coluna: total_price

Cdf.head()

,invoice_id,Branch,City,category,unit_price,quantity,date,time,payment_method,rating,profit_margin,total_price
0,1,WALM003,San Antonio,Health and beauty,74.69,7,05/01/19,13:08:00,Ewallet,9.1,0.48,522.83
1,2,WALM048,Harlingen,Electronic accessories,15.28,5,08/03/19,10:29:00,Cash,9.6,0.48,76.40
2,3,WALM067,Haltom City,Home and lifestyle,46.33,7,03/03/19,13:23:00,Credit card,7.4,0.33,324.31
3,4,WALM064,Bedford,Health and beauty,58.22,8,27/01/19,20:33:00,Ewallet,8.4,0.33,465.76
4,5,WALM013,Irving,Sports and travel,86.31,7,08/02/19,10:37:00,Ewallet,5.3,0.48,604.17


*Alterando o nome das colunas para lower case a fim de facilitar a sql query*

In [72]:
Cdf.columns = Cdf.columns.str.lower() # alterando para lower case
Cdf.columns # verificando 

Index(['invoice_id', 'branch', 'city', 'category', 'unit_price', 'quantity',
       'date', 'time', 'payment_method', 'rating', 'profit_margin',
       'total_price'],
      dtype='object')

**Enviando o dataset para o SQLServer**

*Transformando o dataset em arquivo csv para enviá-lo para o SQLServer*

In [73]:
Cdf.to_csv('Dataset_Walmart_transformed.csv')

*Criação de uma engine sqlAlchemy*

In [74]:
# criando a engine 
from sqlalchemy import create_engine 
engine = create_engine(
    "mssql+pyodbc://@localhost\\SQLEXPRESS/wlt?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes"
)

In [75]:
df = pd.read_csv("Dataset_Walmart_transformed.csv")


In [76]:
# Enviando para o SQL Server

df.to_sql(
    name='Walmart',    
    con=engine,                
    if_exists='replace',       
    index=False                
)

InterfaceError: (pyodbc.InterfaceError) ('28000', '[28000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Login failed for user \'DESKTOP-373OPHL\\Dion\'. (18456) (SQLDriverConnect); [28000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Cannot open database "wlt" requested by the login. The login failed. (4060); [28000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Login failed for user \'DESKTOP-373OPHL\\Dion\'. (18456); [28000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Cannot open database "wlt" requested by the login. The login failed. (4060)')
(Background on this error at: https://sqlalche.me/e/20/rvf5)